# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import nltk
import re
import lightgbm as lgb
from tqdm.notebook import tqdm

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score, make_scorer
from sklearn.pipeline import Pipeline

In [12]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pavelfedorov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pavelfedorov/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/pavelfedorov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pavelfedorov/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except:
    df = pd.read_csv('/Users/pavelfedorov/Portfolio/Project9/toxic_comments.csv')

In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Возьмем только часть данных, чтобы не затягивать обработку

In [5]:
df = df[0:50000]

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50000 non-null  int64 
 1   text        50000 non-null  object
 2   toxic       50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


Напишем код для преобразования текста при помощи модели WordNetLemmatizer

In [7]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [8]:
def clear_text(text):
    resub = re.sub(r'[^a-zA-Z]', ' ', text) 
    return ' '.join(resub.split())

In [9]:
def lemmatize(text):
    text = text.lower()
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(clear_text(text))
    return ' '.join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in word_list])

Проверим что все работает нормально

In [10]:
df['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [13]:
lemmatize(df['text'][0])

'explanation why the edits make under my username hardcore metallica fan be revert they weren t vandalism just closure on some gas after i vote at new york doll fac and please don t remove the template from the talk page since i m retire now'

Запускаем преобразование

In [14]:
tqdm.pandas()

df['lemma_text'] = df['text'].progress_apply(lemmatize)

  0%|          | 0/50000 [00:00<?, ?it/s]

In [15]:
df.head()

,Unnamed: 0,text,toxic,lemma_text
0,0,Explanation\nWhy the edits made under my usern...,0,explanation why the edits make under my userna...
1,1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Создадим фичи и таргет и разделим на обучающую, валидационную и тестовую выборку 8 : 1 : 1

In [16]:
features = df['lemma_text']
target = df['toxic']

In [17]:
features_train, features_valid_test, target_train, target_valid_test  = train_test_split(features, 
                                                                                         target, 
                                                                                         test_size=0.2, 
                                                                                         random_state=42)
features_valid, features_test,target_valid, target_test  = train_test_split(features_valid_test, 
                                                                           target_valid_test, 
                                                                           test_size=0.5, 
                                                                           random_state=42)
print('Признаки обучающей выборки:',features_train.shape)  
print('Целевой признак обучающей выборки:', target_train.shape)
print('Признаки валидационной выборки:',features_valid.shape)  
print('Целевой признак валидационной выборки:', target_valid.shape)
print('Признаки тестовой выборки:',features_test.shape)  
print('Целевой признак тестовой выборки:', target_test.shape)

Признаки обучающей выборки: (40000,)
Целевой признак обучающей выборки: (40000,)
Признаки валидационной выборки: (5000,)
Целевой признак валидационной выборки: (5000,)
Признаки тестовой выборки: (5000,)
Целевой признак тестовой выборки: (5000,)


Преобразуем текст в вектора при помощи метода TfidfVectorizer

In [22]:
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))

In [24]:
features_train_tf_idf = count_tf_idf.fit_transform(features_train.values)
features_valid_tf_idf = count_tf_idf.transform(features_valid.values)
features_test_tf_idf = count_tf_idf.transform(features_test.values)

## Обучение

Обучим 3 модели и подберем параметры на валидационной выборки:

* Линейная регрессия
* Древо решений
* Случайный лес

In [25]:
model_LR = LogisticRegression(solver='liblinear', class_weight = 'balanced', C = 10).fit(features_train_tf_idf,target_train)
prediction_LR = model_LR.predict(features_valid_tf_idf)
print('F1_LR:')
print(f1_score(target_valid, prediction_LR))

F1_LR:
0.7570009033423667


In [26]:
model_DTC = DecisionTreeClassifier(random_state = 42, class_weight = 'balanced').fit(features_train_tf_idf,target_train)
prediction_DTC = model_DTC.predict(features_valid_tf_idf)
print('F1_DTC:')
print(f1_score(target_valid, prediction_DTC))

F1_DTC:
0.6436583261432269


In [27]:
model_RFC = RandomForestClassifier(random_state = 42, class_weight = 'balanced').fit(features_train_tf_idf,target_train)
prediction_RFC = model_RFC.predict(features_valid_tf_idf)
print('F1_RFC:')
print(f1_score(target_valid, prediction_RFC))

F1_RFC:
0.6388888888888888


In [28]:
f1 = make_scorer(f1_score)

In [29]:
# Зададим парметры сетки
parametrs_RFR = { 'n_estimators': range (60, 111, 10),
              'max_depth': range (5,20, 1),
              'min_samples_leaf': range (5,20)
               }

# Определяем модель
RFR_model = RandomForestClassifier(random_state = 42, class_weight = 'balanced',n_jobs=-1)
clf_RFR = RandomizedSearchCV(RFR_model,
                 parametrs_RFR,
                 scoring= f1,
                 random_state = 42)

clf_RFR.fit(features_train_tf_idf,target_train)
print(clf_RFR.best_params_)

{'n_estimators': 100, 'min_samples_leaf': 10, 'max_depth': 19}


In [30]:
pipe_rf = Pipeline([('count_tf_idf',  TfidfVectorizer()),
                    ('RFR_model', RandomForestClassifier(random_state=42))])

In [31]:
parametrs_pipeline_RFR = { 'RFR_model__n_estimators': range (60, 111, 10),
                           'RFR_model__max_depth': range (5,20, 1),
                           'RFR_model__min_samples_leaf': range (5,20)
                           }

rf = RandomizedSearchCV(pipe_rf,
            parametrs_pipeline_RFR,
            scoring=f1,
            cv=3, 
            n_jobs=-1)
rf.fit(features_train,target_train)
print(rf.best_params_)

{'RFR_model__n_estimators': 90, 'RFR_model__min_samples_leaf': 10, 'RFR_model__max_depth': 15}


In [32]:
# Зададим парметры сетки
parametrs_DTR = {'max_depth': range (5,20, 1),
              'min_samples_split': range (5,20)
               }

# Определяем модель
DTR_model = DecisionTreeClassifier(random_state = 42, class_weight = 'balanced')
clf_DTR = RandomizedSearchCV(DTR_model,
                 parametrs_DTR,
                 scoring=f1,
                 random_state = 42)

clf_DTR.fit(features_train_tf_idf,target_train)
print(clf_DTR.best_params_)

{'min_samples_split': 15, 'max_depth': 16}


In [33]:
pipe_dt = Pipeline([('count_tf_idf',  TfidfVectorizer()),
                    ('DTC_model', DecisionTreeClassifier(random_state=42))])

In [34]:
parametrs_pipeline_DT = { 'DTC_model__max_depth': range (5,20, 1),
                           'DTC_model__min_samples_split': range (5,20)
                           }

dt = RandomizedSearchCV(pipe_dt,
            parametrs_pipeline_DT,
            scoring=f1,
            cv=3, 
            n_jobs=-1)
dt.fit(features_train,target_train)
print(dt.best_params_)

{'DTC_model__min_samples_split': 9, 'DTC_model__max_depth': 18}


In [35]:
final_model_RFR = RandomForestClassifier(random_state = 42, 
                                            class_weight = 'balanced',
                                            n_estimators = 80,
                                            min_samples_leaf = 17,
                                            max_depth = 19)
final_model_RFR.fit(features_train_tf_idf,target_train)
predicted_RFR_final = final_model_RFR.predict(features_valid_tf_idf)
print(f1_score(target_valid , predicted_RFR_final))

0.41674249317561424


In [36]:
final_model_DTR = DecisionTreeClassifier(random_state = 42, 
                                            class_weight = 'balanced',
                                            min_samples_split = 7,
                                            max_depth = 18)
final_model_DTR.fit(features_train_tf_idf,target_train)
predicted_DTR_final = final_model_DTR.predict(features_valid_tf_idf)
print(f1_score(target_valid , predicted_DTR_final))

0.6305525460455039


Линейная регрессия лучше всего себя показала, поэтмоу используем ее на тесте

In [37]:
model_test_LR = LogisticRegression(solver='liblinear', class_weight = 'balanced', C = 10).fit(features_train_tf_idf,target_train)
prediction_LR = model_test_LR.predict(features_test_tf_idf)
print('F1_LR:')
print(f1_score(target_test, prediction_LR))

F1_LR:
0.7532467532467534


## Выводы

В рузльатет проделанной работы было достигнуто , при помощи модели Линейной регресси, значение необходимой метрики F1 выше 0.75 на тестовой выборке. Для токонизации текста использовался метод TfidfVectorizer, предварительно текст был преобразован при помощи метода WordNetLemmatizer